# Benchmark

We use a list of public datasets to benchmark all the algorithms in StreamAD. Thanks!

1. [AIOps Challenge public dataset for KPI anomaly detection](https://github.com/NetManAIOps/KPI-Anomaly-Detection)
2. 

Firstly, we create a dictionary of the datasets, and download those public datasets.

In [ ]:
import os
import subprocess
import pandas as pd
import numpy as np
from time import perf_counter
from streamad.util import StreamGenerator, CustomDS, plot
from streamad.model import SpotDetector
from tqdm import tqdm
from streamad.evaluate import (
    NumentaAwareMetircs,
    PointAwareMetircs,
    SeriesAwareMetircs,
)


In [ ]:
path = './streamad-benchmark-dataset'
if not os.path.exists(path):
    os.makedirs(path)

# Dataset 1

# Git clone and unzip the files
subprocess.check_call(["git", "clone", "--depth=1", "https://github.com/NetManAIOps/KPI-Anomaly-Detection.git", path +'/dataset1'])
subprocess.check_call(["unzip", "streamad-benchmark-dataset/dataset1/Finals_dataset/phase2_ground_truth.hdf.zip", "-d","streamad-benchmark-dataset/dataset1/Finals_dataset/"])
subprocess.check_call(["unzip", "streamad-benchmark-dataset/dataset1/Finals_dataset/phase2_train.csv.zip", "-d","streamad-benchmark-dataset/dataset1/Finals_dataset/"])

In [ ]:
dataset = "dataset1"
# For preliminary datasets.
df_pre_train = pd.read_csv(path + '/dataset1/Preliminary_dataset/train.csv')
df_pre_train_groups = df_pre_train.groupby("KPI ID")
keys = df_pre_train_groups.groups.keys()

benchmark_items = [
    "Dataset",
    "Key",
    "Size(#)",
    "Time(s)",
    "PointAwarePrecision",
    "PointAwareRecall",
    "PointAwareFbeta",
    "SeriesAwarePrecision",
    "SeriesAwareRecall",
    "SeriesAwareFbeta",
    "NumentaAwarePrecision",
    "NumentaAwareRecall",
    "NumentaAwareFbeta",
]

benchmark_df = pd.DataFrame(columns=benchmark_items)


for key in keys:
    df = df_pre_train_groups.get_group(key)
    ds = df[["timestamp", "value", "label"]]
    label = df["label"]

    ds = CustomDS(ds, label)
    stream = StreamGenerator(ds.data)
    model = SpotDetector(window_len=200)

    scores = []

    start_time = perf_counter()
    for x in tqdm(stream.iter_item(), total=len(ds.data)):
        score = model.fit_score(x)
        scores.append(score)

    time = perf_counter() - start_time

    benchmark_values = [dataset, key, len(ds.data), time]

    label = ds.label
    for metric in [
        PointAwareMetircs(),
        SeriesAwareMetircs(),
        NumentaAwareMetircs(),
    ]:

        scores = np.nan_to_num(np.array(scores, dtype=float), nan=0)
        benchmark_values.extend(list(metric.evaluate(label, scores)))

    benchmark_df.loc[len(benchmark_df)] = benchmark_values

In [7]:
benchmark_df.to_csv('./benchamark_results.csv', index=False)

In [8]:
benchmark_df

,Dataset,Key,Size(#),Time(s),PointAwarePrecision,PointAwareRecall,PointAwareFbeta,SeriesAwarePrecision,SeriesAwareRecall,SeriesAwareFbeta,NumentaAwarePrecision,NumentaAwareRecall,NumentaAwareFbeta
0,dataset1,02e99bd4f6cfb33f,128562,211.259536,0.571429,0.001137,0.002270,0.900000,0.001765,0.003523,0.571429,0.002022,0.004030
1,dataset1,046ec29ddf80d62e,8784,6.627660,1.000000,0.012500,0.024691,1.000000,0.012500,0.024691,1.000000,0.012500,0.024691
2,dataset1,07927a9a18fa19ae,10960,28.547375,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,dataset1,09513ae3e75778a3,128971,751.831710,0.416667,0.021008,0.040000,0.714286,0.051587,0.096225,0.416667,0.051587,0.091808
4,dataset1,18fbb1d5a5dc099d,129128,110.694167,0.490506,0.019713,0.037902,0.641611,0.139135,0.228681,0.490506,0.223652,0.307223
5,dataset1,1c35dbf57f55f5e4,128853,74.565472,0.669291,0.008872,0.017511,0.891934,0.115922,0.205178,0.669291,0.137973,0.228783
6,dataset1,40e25005ff8992bd,100254,48.765101,1.000000,0.120795,0.215553,1.000000,0.349425,0.517888,1.000000,0.349425,0.517888
7,dataset1,54e8a140f6237526,8248,13.912780,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,dataset1,71595dd7171f4540,147668,155.893854,1.000000,0.070979,0.132550,1.000000,0.281647,0.439507,1.000000,0.281754,0.439638
9,dataset1,769894baefea4e9e,8784,2.502098,1.000000,0.111111,0.200000,1.000000,0.111111,0.200000,1.000000,0.111111,0.200000


In [ ]:
# pd.read_hdf("streamad-benchmark-dataset/dataset1/Finals_dataset/phase2_ground_truth.hdf")